# Estudo sobre Doação de sangue

## 1. Sobre o Projeto

Nesse projeto, buscamos pesquisar e entender a similaridade entre doadores de sangue e a construção de um algoritmo capaz de predizer a probabilidade de uma pessoa ser doadora de sangue.
Para isso, usamos a base de dados Predicting Blood Donor Availability *(disponíel em : https://www.kaggle.com/datasets/sumedh1507/blood-donor-dataset?resource=download)* para elaboração desse projeto.

Essa base de dados possui 10 mil entradas, com os seguintes atributos:
- Id de doador
- Nome
- Email
- Senha
- Telefone
- Cidade
- Tipo sanguíneo
- Disponibilidade
- Meses desde a primeira doação
- Número de doações
- Pints (500ml) doados
- Data de criação

## 1.1 Metodologia

Como nosso objetivo principal é chegarmos a uma predição de uma nova entrada, irems estabeler algumas regras para execução desse estudo.

Para nos auxiliar com o estudo, iremos focar apenas nas propriedades: Tipo sanguíneo, Disponibilidade, Meses desde a primeira doação, número de doações e data de criação.

## 1.2 Agrupamento

Para auxiliar na predição do nosso modelo, agruparemos nossos dados em dois grupos:
- Doadores ativos
- Doadores inativos

Os ativos são doadores que desde a sua primeira doação, fizeram pelo menos uma doação por ano. Já os inativos, são todos aqueles que doaram menos de 1x por ano desde seu registro.

Isso será importante para classificação dos doadores na nossa base de dados

# 2. Algoritmo do projeto

### Variáveis globais

In [3]:
DATASET_PATH = "./dataset/blood_donor_dataset.csv"

## 2.1 Importação de bibliotecas

In [4]:
import pandas as pd

## 2.2 Atribuição da base de dados

In [ ]:
df = pd.read_csv(DATASET_PATH)

        donor_id              name                          email    password  \
0     1b8b4c828a      Norma Fisher            ysullivan@yahoo.com  P5hX6Syg*A   
1     a6a3f7fe55     Eugene Bowman       hramos@brown-sellers.com  b6Wq6Cqz@5   
2     cee6478144  Sheri Bolton DDS          jasmine85@hotmail.com  $9RB51m#s7   
3     5986bc55f9        Kelly Boyd  pattylawrence@riley-hayes.com  @5Tdg(4kCm   
4     93246afe6c     Taylor Guzman       martinezjacob@wilson.com  x!)9Ag@J*7   
...          ...               ...                            ...         ...   
9995  700aeefbf1      Bradley Pham           jessica49@turner.com  ^@d9SMqSa1   
9996  487b342992    Richard Harris              megan51@gmail.com  D1OAZg5i)g   
9997  ddf4485d67        Autumn Kim       curtisdonald@wilson.info  3wU2fua(#7   
9998  4256813ba5     Janice Murphy   jenningscristian@hotmail.com  &8zDeXl)b(   
9999  5452d8afd3     Edward Graham               oyates@yahoo.com  ud(J5UjHh9   

     contact_number       c

# 3. Pré processamento de dados

Como dito anteriormente, iremos remover alguns dados irrelevantes para a parametrização do nosso (nome, telefone, cidade, etc...). Acreditamos que essas propriedades não impactam/ não fazem sentido para o problema proposto.

In [ ]:
df.drop("donor_id", axis="columns" )